In [ ]:
import os
import cobra
import scripts.io
import scripts.gen_model
import scripts.reaction_utils
import scripts.metabolite_utils
import math
import numpy as np
import importlib
import thermo_flux.io.load_sbml


from thermo_flux.tools.drg_tools import reaction_balance

In [ ]:
INPUT_MODEL = "datafiles/model.xlsx"
INPUT_KEGGS = "datafiles/ecoli_kegg_id.csv"
INPUT_REED = "regression/reed.csv"
INPUT_INCHI = "regression/InChIs.csv"
INPUT_GAMS = "regression/model_Ecoli_from-gams.xlsx"
INPUT_EXP_DATA = "regression/allPhysioData_formatted_forGSM_20230831.csv"
INPUT_EXP_CONC = "regression/allConcRange_20230912.csv"
INPUT_METABOLOMICS = "regression/metabolomics-Kochanowski_20230925.csv"

MODEL_NAME = "ecoli"

CONDITIONS = ["WT-Glc_I", "WT-Ace_I"]

INCLUDE_CO2 = True
INCLUDE_O2 = True
ALLOW_OTHER_EXCRETION = False
RELAX_EXP_FLUX_BOUNDS = 2.0

In [ ]:
import pandas as pd

def add_gas_constraint(model, condition, phys_data, epsilon=0.25):

    # set co2/o2 fluxes in correct directions
    model.reactions.EX_co2.lower_bound = 0.0
    model.reactions.EX_co2.upper_bound = 100.0
    
    model.reactions.EX_o2.lower_bound = -100.0
    model.reactions.EX_o2.upper_bound = 0.0

    reg_data = pd.read_csv(phys_data)

    reg_data.set_index(["cond", "rxn"], inplace=True) 

    reg_data_gas = reg_data.swaplevel().copy()
    reg_data_gas = reg_data_gas.loc[["EX_co2", "EX_o2"]]
    reg_data_gas = reg_data_gas.swaplevel()

    co2_mean = reg_data_gas.loc[(condition, "EX_co2"), "mean"]
    o2_mean = reg_data_gas.loc[(condition, "EX_o2"), "mean"]

    co2_sd = reg_data_gas.loc[(condition, "EX_co2"), "sd"]
    o2_sd = reg_data_gas.loc[(condition, "EX_o2"), "sd"]


    if model.m is not None:
        co2_idx = model.reactions.index("EX_co2")
        o2_idx = model.reactions.index("EX_o2")

        # as we have a mean and sd for the o2/co2 fluxes I am using a ratio range corresponding to the lower/upper range of the mean + sd
        R_upper = abs( (co2_mean + co2_sd) / (o2_mean + o2_sd) )
        R_lower = abs( (co2_mean - co2_sd) / (o2_mean - o2_sd) )

        # add constraints for the lower range of the ratio and upper range of the ratio
        # also, instead of saying that it needs to be 0, I set it to be within a specific range (the epsilon)
        # I noticed that the model becomes infeasible for Glc/Ace with an epsilon of 0.05 (5%), with 0.25 (25%) it is feasible
        model.m.addConstr( model.mvars["v"][0][co2_idx] + (R_lower * model.mvars["v"][0][o2_idx]) >= -epsilon, name=f"RQ_ratio_lower_{condition}")
        model.m.addConstr( model.mvars["v"][0][co2_idx] + (R_upper * model.mvars["v"][0][o2_idx]) <= epsilon, name=f"RQ_ratio_upper_{condition}")
    else:
        "No gurobi model, add TFBA variables first."

In [ ]:
def setup_model(condition):
    # Setup model with only removing blocked reactions
    # No compression or cofactor loops removed etc.
    tmodel = scripts.gen_model.gen_model(MODEL_NAME, INPUT_MODEL, INPUT_KEGGS, INPUT_REED, INPUT_INCHI, INPUT_GAMS, "", True, True)
    tmodel = scripts.gen_model.apply_physio_data(tmodel, condition, INPUT_EXP_DATA, INPUT_EXP_CONC, INPUT_METABOLOMICS, INPUT_GAMS, RELAX_EXP_FLUX_BOUNDS, INCLUDE_CO2, INCLUDE_O2, allow_other_excr=False, output_log="", flux_limit=100)

    blocked_p = scripts.reaction_utils.list_blocked_reactions(tmodel, "core", "", 1, False)
    print(len(blocked_p))

    tmodel.remove_reactions(blocked_p, remove_orphans=True)
    scripts.metabolite_utils.remove_orphan_metabolites(tmodel)
    for rxn in tmodel.reactions:
        reaction_balance(rxn, balance_charge=True, balance_mg=False)
    tmodel.update_thermo_info(fit_unknown_dfG0=True)

    return tmodel

In [ ]:
tmodel_glc = setup_model(CONDITIONS[0])
tmodel_glc.m = None
tmodel_glc.objective = tmodel_glc.reactions.biomass_EX
tmodel_glc.add_TFBA_variables()

add_gas_constraint(tmodel_glc, "WT-Glc_I", INPUT_EXP_DATA)

In [ ]:
tmodel_ace = setup_model(CONDITIONS[1])
tmodel_ace.m = None
tmodel_ace.objective = tmodel_ace.reactions.biomass_EX
tmodel_ace.add_TFBA_variables()

add_gas_constraint(tmodel_ace, "WT-Ace_I", INPUT_EXP_DATA)

In [ ]:
# Write one model for all concentration TVA's scenarios
mets_glc = scripts.reaction_utils.tfva_run_scenarios_one_model_mets(tmodel_glc, "ecoli", "WT_Glc", "mets", "", False, False, ONLY_WRITE=True)
mets_ace = scripts.reaction_utils.tfva_run_scenarios_one_model_mets(tmodel_ace, "ecoli", "WT_Ace", "mets", "", False, False, ONLY_WRITE=True)

In [ ]:
# Write one mps file containign 2 scenarios for each reaction for the models
scripts.reaction_utils.tfva_run_scenarios_one_model(tmodel_glc, "ecoli", "WT_Glc", "results/WT_Glc/parallel/", REMOVE_BLOCKED=False, APPLY_FVA=False, RUN_DIRECTLY=True, ONLY_WRITE=True, mipgap=0.01)
scripts.reaction_utils.tfva_run_scenarios_one_model(tmodel_ace, "ecoli", "WT_Ace", "results/WT_Ace/parallel/", REMOVE_BLOCKED=False, APPLY_FVA=False, RUN_DIRECTLY=True, ONLY_WRITE=True, mipgap=0.01)